In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2" #no annoyiing messages
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import mnist

In [2]:
# Make sure we don't get any GPU errors
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

### Importing Data from MNIST available on tf keras datasets

In [11]:
(x_train, y_train),(x_test,y_test) = mnist.load_data()
x_train = x_train.reshape(-1,28*28).astype('float32')/255.0
x_test = x_test.reshape(-1,28*28).astype('float32')/255.0

## What we are trying to do
We will create custom layers using subclassing like API of Keras

In [12]:
class MyModel(keras.Model):
    def __init__(self, num_classes=10):
        super(MyModel, self).__init__()
        self.dense1 = layers.Dense(64)
        self.dense2 = layers.Dense(num_classes)
        
    def call(self, input_tensor):
        x = tf.nn.relu(self.dense1(input_tensor))
        return self.dense2(x)

In [13]:
model = MyModel()
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(),
    metrics = ['accuracy']
)

In [15]:
model.fit(x_train,y_train, epochs = 3, batch_size=32, verbose=2)
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

Epoch 1/3
1875/1875 - 2s - loss: 0.0809 - accuracy: 0.9761
Epoch 2/3
1875/1875 - 2s - loss: 0.0665 - accuracy: 0.9797
Epoch 3/3
1875/1875 - 2s - loss: 0.0562 - accuracy: 0.9826
313/313 - 0s - loss: 0.0866 - accuracy: 0.9720


[0.08662107586860657, 0.972000002861023]

### Now we will create our Dense layer by ourselves  

In [17]:
class Dense(layers.Layer):
    def __init__(self, units, input_dim):
        super(Dense,self).__init__()
        #now, we will add weights
        self.w = self.add_weight(
            name='w',
            shape=(input_dim,units),
            initializer='random_normal',
            trainable=True
        )
        
        #add bias
        self.b = self.add_weight(
            name='b', shape = (units,), initializer ='zeros', trainable=True
        )

    def call(self, inputs):
        # we will perform the basic operation of a  neutral net, W.X + b and return
        return tf.matmul(inputs,self.w) + self.b

Now, we will remake mymodel with OWN dense layers

In [19]:
class MyModel_withDense(keras.Model):
    def __init__(self, num_classes=10):
        super(MyModel_withDense, self).__init__()
        self.dense1 = Dense(units = 64, input_dim = 28*28)
        self.dense2 = Dense(units = 10, input_dim = 64)
        
    def call(self, input_tensor):
        x = tf.nn.relu(self.dense1(input_tensor))
        return self.dense2(x)
    
model = MyModel_withDense()
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(),
    metrics = ['accuracy']
)

model.fit(x_train,y_train, epochs = 3, batch_size=32, verbose=2)
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

Epoch 1/3
1875/1875 - 1s - loss: 0.3519 - accuracy: 0.9036
Epoch 2/3
1875/1875 - 1s - loss: 0.1664 - accuracy: 0.9514
Epoch 3/3
1875/1875 - 1s - loss: 0.1151 - accuracy: 0.9662
313/313 - 0s - loss: 0.1118 - accuracy: 0.9663


[0.11184829473495483, 0.9663000106811523]

It works!
But there is still something, We have to specify the input dimension for each Dense layer. We will now make it so that this is worked out by the Dense Layer itself. We will also implement the Relu Layer

In [22]:
class MyRelu(layers.Layer):
    def __init__(self):
        super(MyRelu, self).__init__()
        
    def call(self,x):
        return tf.math.maximum(x,0)

In [20]:
class Dense_Lazy(layers.Layer):
    def __init__(self, units):
        super(Dense_Lazy,self).__init__()
        self.units = units

    def build(self, input_shape):
        # we will make the weights here only
        self.w = self.add_weight(
            name='w',
            shape=(input_shape[-1],self.units),
            initializer='random_normal',
            trainable=True
        )
        #add bias
        self.b = self.add_weight(
            name='b', shape = (self.units,), initializer ='zeros', trainable=True
        )
    
    def call(self, inputs):
        # we will perform the basic operation of a  neutral net, W.X + b and return
        return tf.matmul(inputs,self.w) + self.b

In [23]:
class MyModel_withDense(keras.Model):
    def __init__(self, num_classes=10):
        super(MyModel_withDense, self).__init__()
        self.dense1 = Dense_Lazy(64)
        self.dense2 = Dense_Lazy(10)
        self.relu = MyRelu()
        
    def call(self, input_tensor):
        x = self.relu(self.dense1(input_tensor))
        return self.dense2(x)

model = MyModel_withDense()
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(),
    metrics = ['accuracy']
)

model.fit(x_train,y_train, epochs = 3, batch_size=32, verbose=2)
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

Epoch 1/3
1875/1875 - 2s - loss: 0.3393 - accuracy: 0.9076
Epoch 2/3
1875/1875 - 2s - loss: 0.1605 - accuracy: 0.9534
Epoch 3/3
1875/1875 - 2s - loss: 0.1170 - accuracy: 0.9657
313/313 - 0s - loss: 0.1083 - accuracy: 0.9684


[0.10829201340675354, 0.9684000015258789]